### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.14.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.14.1-alpha"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#!import Secrets.cs

using Microsoft.SemanticKernel;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    .Build();

### Initialize Chat Completion service

In [ ]:

var chatGPT = kernel.GetRequiredService<IChatCompletionService>();


### Initialize chat history

In [ ]:
using Microsoft.SemanticKernel.ChatCompletion;

var systemMessage = "You are a librarian, expert about books";
var chatHistory = new ChatHistory(systemMessage);

// or in a more compact form
//var chatHistory = new ChatHistory("You are a librarian, expert about books");


### Let's rebuild a chat history by adding some user and assistants messages to it.
### Please notice that we have two types of messages below: user and assistant messsages.

In [ ]:
chatHistory.AddUserMessage("Hi, I'm looking for book suggestions");

### 🤖 And now adding an assistant message (in fact, a response from model)

In [ ]:

chatHistory.AddAssistantMessage("Great, I'd be happy to help with book suggestions. To better assist you, could you let me know what genres or types of books you're interested in? Some further details such as your favorite books or authors, themes you enjoy, or what mood you're in for reading could help me provide more tailored recommendations.");

### 👩‍💻 Add one more user message

In [ ]:

chatHistory.AddUserMessage("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?");

### Let's inspect the chat history so far

In [ ]:

foreach (var message in chatHistory)
{
    Console.WriteLine($"{message.Role} > {message.Content}");
}

### 🤖🏃‍♂️Now we ask the model for a response (we can notice the reply from the assistant message)

In [ ]:
var reply = await chatGPT.GetChatMessageContentAsync(chatHistory);
chatHistory.Add(reply);

$"{chatHistory[^1].Role} > {chatHistory[^1].Content}"

### 🚰 🤖🏃‍♂️ We can ask the model for a response in streaming flavour

In [ ]:
Console.Write($"{Azure.AI.OpenAI.ChatRole.Assistant} > ");
await foreach (var content in chatGPT.GetStreamingChatMessageContentsAsync(chatHistory))
{
    Console.Write(content);
}